In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
#limiting gpu usage
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu,True)

In [ ]:
#importing model libraries
import numpy as np

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

from tensorflow.keras.layers import GRU,SimpleRNN,LSTM,BatchNormalization,TimeDistributed,Bidirectional
from keras.optimizers import SGD, Adam

In [ ]:
# Define Keras model

N_HIDDEN = 256

N_CONTEXT = 40 + 1 # 20 + 1 + 20 frames
N_INPUT = 13 # mfcc features

N_OUTPUT = 100  # 50 * 2


model = Sequential()
model.add(TimeDistributed(Dense(N_HIDDEN), input_shape=(N_CONTEXT, N_INPUT)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(TimeDistributed(Dense(N_HIDDEN)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(TimeDistributed(Dense(N_HIDDEN)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(GRU(N_HIDDEN, return_sequences=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(Dense(N_OUTPUT))
model.add(Activation('linear'))

print(model.summary())

In [ ]:
# data preparation
X = np.load("/content/drive/MyDrive/dataset/mfccs_file.npy")
Y = np.load("/content/drive/MyDrive/dataset/keypoints_file_v2.npy")

N_train = int(len(X)*0.9)
N_validation = len(X) - N_train

# Split on training and validation
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=N_validation)

print(X_train.shape)
print(Y_train.shape)

In [ ]:
# compile and train model
EPOCHS = 4
BATCH_SIZE = 64

optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
model.compile(loss='mean_squared_error', optimizer=optimizer)

hist = model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(X_validation, Y_validation))

In [ ]:
# save model
model.save("model/gesture.h5")
model.save_weights("model/weights")

!zip -r /content/model.zip /content/model

In [ ]:
# loading model 
from tensorflow.keras.models import load_model
model_loaded = tf.keras.Model()
model_loaded = load_model("gesture.h5")
model_loaded.summary()

In [ ]:
# prediction 
X = np.load('file_name.npy')
predicted = np.array(model.predict(X))
print(predicted.shape)